# Computer Assignment #1
## Parsa Alizadeh 810101572
## Implementation of Informed and Uninformed Search Algorithms and Solving a Problem with Them.
### Problem Description
The problem is to find the least moves to put boxes in the designated places.

## Question number 1:
 In this problem, what can be considered as the definition of a state? The most obvious definition is the map of the warehouse at any given moment. What is the issue with this definition, and how can it be improved?

## Answer: 
The state can be defined as the combination of: 
* The player's position.
* The positions of all boxes. 
* The positions of portals (if they affect the state).
<br>
The issue with using the entire map as the state is that it can be inefficient and redundant. The map includes static elements like walls and goals, which do not change during the game. Storing the entire map for each state consumes unnecessary memory and slows down the search.
For improving the states we can use only dynamic elements and do not store the whole map. The things I mentioned above are the dynamic elements.

In the code, the state is defined as a tuple containing:
* The player's position: ```(row, col).```
* The positions of all boxes: ```((row1, col1), (row2, col2), ...).```<br>
for example: 
```state = (player_pos, boxes)```
<br>

## Question number 2:
Based on your definition of the state, how would you define actions?
## Answer:
Actions or State transitions are defined by the player's movement in four directions (U, D, L, R). Each move updates: 
* The player's position, 
* The position of any box the player pushes, 
* The state of portals (if the player or a box enters a portal).

## Question number 3:
Explain how you would model the problem, including the definition of actions, states, goal states, etc.

## Answer:
The problem is modeled as follows in the code:

* State: A tuple ```(player_pos, boxes)```, where:
<br>
```player_pos```: The player's current position (row, col).
<br>
```boxes```: A tuple of positions for all boxes ((row1, col1), (row2, col2), ...).

* Actions: The player can move in four directions (U, D, L, R).
* Goal State: A state where all boxes are on their corresponding goals. This is checked using ```game.is_game_won()```.
* Initial State: The starting positions of the player and boxes, obtained from the map.
* Transitions: For each action, a new state is generated by updating the player's position and, if applicable, the box's position.

## Question number 4:
How can this problem be improved, and how can the search space be optimized? Should we expand all possible states in the next step of the search algorithms? What are the implications of doing so? What measures can be taken to improve this problem?

## Answer:
### Improvements in the Code:
<br>
* The code uses a visited set to avoid revisiting the same state, which reduces redundant exploration.
<br>
* For A*, a heuristic (Manhattan distance) is used to prioritize states closer to the goal.

### Expanding All States:
* Expanding all possible states at each step can lead to exponential growth in the search space, making the algorithm slow and memory-intensive.
<br>
* This is avoided in the code by only generating valid states (e.g., states where the player or boxes do not collide with walls or other boxes).

### Optimization Measures:
* Heuristics: Used in A* to guide the search.
* State Caching: The ```visited``` set ensures that each state is explored only once.
* Iterative Deepening: Used in IDS to balance memory usage and completeness.

In [ ]:
from game import Game
import time
import os
import re
from collections import deque
import heapq
import time
import pandas as pd
import time
from multiprocessing import Process, Queue
import numpy as np
from scipy.optimize import linear_sum_assignment



## Load maps

In [3]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## Tutorial

In [4]:
print("This is an example of the game map:")
map = load_map(2)
game = Game(map)
game.display_map()

This is an example of the game map:
W	P1	H	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [5]:
game.get_box_locations()

[(2, 3), (3, 2), (4, 3)]

In [6]:
game.get_goal_locations()

[(1, 3), (3, 1), (5, 3)]

In [7]:
game.get_player_position()

(0, 2)

- W : Wall
- H : Human
- B : Box
- P : Portal
- G : Goal

In [8]:
for direction in ['U', 'D', 'R', 'L']:
    result = game.apply_move(direction)
    print(f"Move {direction} is valid: {result}")
    if result:
        game.display_map()

Move U is valid: False
Move D is valid: False
Move R is valid: False
Move L is valid: True
W	P1	.	W	W	W	W
W	W	W	G1	W	W	W
W	W	W	B1	W	W	W
W	G2	B2	H	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [9]:
game.apply_move('U')
game.display_map()

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	H	W	W	W
W	G2	B2	.	P1	W	W
W	W	W	B3	W	W	W
W	W	W	G3	W	W	W
W	W	W	W	W	W	W


In [10]:
game.apply_moves(['D', 'L', 'R', 'D']) 
game.display_map()
print("Is game won?", game.is_game_won())

W	P1	.	W	W	W	W
W	W	W	G1/B1	W	W	W
W	W	W	.	W	W	W
W	G2/B2	.	.	P1	W	W
W	W	W	H	W	W	W
W	W	W	G3/B3	W	W	W
W	W	W	W	W	W	W
Is game won? True


## Solvers

### Question number 5: 
explain each algorithms and mention their andvantages and disadvantages.
<br>
This question in answered in each part of below algorithms.

### BFS

This algorithm Explores all states level by level. The advantage of this algorythm is that it Guarantees the shortest path (optimal solution). Its disadvantage is that it has High memory usage for large state spaces.

In [ ]:
def solver_bfs(game_map):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    visited = set()
    queue = deque([(initial_state, [])])  

    while queue:
        (player_pos, boxes), path = queue.popleft()

        if (player_pos, boxes) in visited:
            continue
        visited.add((player_pos, boxes))

        game.set_player_position(player_pos)
        game.set_box_positions(list(boxes))

        if game.is_game_won():
            return path, len(visited)

        for direction in ['U', 'D', 'L', 'R']:
            # Avoid reversing the last move 
            if path and is_opposite_direction(direction, path[-1]):
                continue

            if game.apply_move(direction):
                new_player_pos = game.get_player_position()
                new_boxes = tuple(game.get_box_locations())

                if game.is_game_won():
                    return path + [direction], len(visited)

                # Add to queue if not visited
                if (new_player_pos, new_boxes) not in visited:
                    queue.append(((new_player_pos, new_boxes), path + [direction]))

            # Reset game state for next move
            game.set_player_position(player_pos)
            game.set_box_positions(list(boxes))

    return None, len(visited)

def is_opposite_direction(dir1, dir2):
    opposites = {'U': 'D', 'D': 'U', 'L': 'R', 'R': 'L'}
    return dir1 == opposites.get(dir2, None)

### DFS

This algorithm Explores states deeply before backtracking. The advantage of this algorythm is that it has Low memory usage. Its disadvantage is that it is Not optimal. It can get stuck in infinite loops.
#### Question number 6:
 Question:
 <br>
  What is the issue with DFS, and why is it still used?
  <br>
  
 Answer:
* The issue of DFS is the it is not optimal and can get stuck in infinite loops or long paths.
* The reason Why it is Used is that DFS is simple to implement, uses less memory than BFS, and can be effective for problems where the solution is deep in the search tree.


In [ ]:
def solver_dfs(game_map):
   
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    stack = [(initial_state, [])]  
    visited = set()

    while stack:
        (player_pos, boxes), path = stack.pop()

        if (player_pos, boxes) in visited:
            continue
        visited.add((player_pos, boxes))

        game.set_player_position(player_pos)
        game.set_box_positions(list(boxes))

        if game.is_game_won():
            return path, len(visited)

        for direction in ['U', 'D', 'L', 'R']:
            # Avoid reversing the last move 
            if path and is_opposite_direction(direction, path[-1]):
                continue

            if game.apply_move(direction):
                new_player_pos = game.get_player_position()
                new_boxes = tuple(game.get_box_locations())

                if game.is_game_won():
                    return path + [direction], len(visited)

                # Add to stack if not visited
                if (new_player_pos, new_boxes) not in visited:
                    stack.append(((new_player_pos, new_boxes), path + [direction]))

            # Reset game state for next move
            game.set_player_position(player_pos)
            game.set_box_positions(list(boxes))

    return None, len(visited)  # No solution found

def is_opposite_direction(dir1, dir2):
  
    opposites = {'U': 'D', 'D': 'U', 'L': 'R', 'R': 'L'}
    return dir1 == opposites.get(dir2, None)

### IDS

This algorithm Combines BFS and DFS by performing DFS with increasing depth limits. The advantage of this algorythm is that it is Optimal like BFS, but uses less memory.Its disadvantage is that it is Slower due to repeated exploration of states.

In [ ]:
def solver_ids(game_map):
    def depth_limited_search(state, path, depth, visited, game):
        if depth == 0:
            return None

        player_pos, boxes = state
        game.set_player_position(player_pos)
        game.set_box_positions(list(boxes))

        if game.is_game_won():
            return path

        state_key = (player_pos, frozenset(boxes))
        if state_key in visited:
            return None
        visited.add(state_key)

        for direction in ['U', 'D', 'L', 'R']:  
            prev_player_pos = game.get_player_position()
            prev_boxes = tuple(game.get_box_locations())

            if game.apply_move(direction):
                new_state = (game.get_player_position(), tuple(game.get_box_locations()))
                result = depth_limited_search(new_state, path + [direction], depth - 1, visited, game)
                if result is not None:
                    return result

            # Undo the move
            game.set_player_position(prev_player_pos)
            game.set_box_positions(list(prev_boxes))

        return None

    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    depth = 0

    while True:
        visited = set()
        result = depth_limited_search(initial_state, [], depth, visited, game)
        if result is not None:
            return result, len(visited)
        depth += 1


### A*

This algorithm Uses a heuristic to prioritize states closer to the goal. The advantage of this algorythm is that it is Optimal if the heuristic is admissible. Its disadvantage is that it Requires a good heuristic or else it will not perform well.

### How is the heuristic calculated?
The heuristic estimates the remaining cost to reach the goal state. We use a Cost matrix that Rows represent where the  boxesare and Columns represent goal positions. Then we use hungarian algorithm linear_sum_assignment from SciPy. Instead of just summing the individual Manhattan distances, this approach finds the best possible combination of assignments. Manhattan Distance assigns each box to a goal independently, which may lead to a bad solution.
but Hungarian Algorithm assigns boxes optimally, minimizing the overall cost.

### Weighted A*
This algorithm is Similar to A* but introduces a weight to prioritize the heuristic over the path cost. Its advantage is that it is  Faster than A*. But as a disadvantage it may not guarantee optimality.

In [ ]:
class GameWrapper:
    def __init__(self, game_map):
        self.game = Game(game_map)
        self.goals = tuple(self.game.get_goal_locations())

    def heuristic(self, boxes):
        if len(boxes) != len(self.goals):
            return float('inf')  # Impossible case

        #Manhattan distance 
        cost_matrix = np.zeros((len(boxes), len(self.goals)))
        for i, box in enumerate(boxes):
            for j, goal in enumerate(self.goals):
                cost_matrix[i, j] = abs(box[0] - goal[0]) + abs(box[1] - goal[1])

        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        return cost_matrix[row_ind, col_ind].sum()

    def is_goal_state(self, boxes):
        return set(boxes) == set(self.goals)


def solver_astar(game_map, weight=1):
    game = GameWrapper(game_map)
    initial_state = (game.game.get_player_position(), tuple(game.game.get_box_locations()))

    priority_queue = []
    heapq.heappush(priority_queue, (0, 0, initial_state))  # (priority, tie-breaker, state)

    visited = set()
    g_scores = {initial_state: 0}  # Cost to reach to each state
    parents = {}  # Store state transitions for path reconstruction
    counter = 0  #  counter

    while priority_queue:
        _, _, (player_pos, boxes) = heapq.heappop(priority_queue)

        if (player_pos, boxes) in visited:
            continue
        visited.add((player_pos, boxes))

        if game.is_goal_state(boxes):
            # Reconstruct path
            path = []
            while (player_pos, boxes) in parents:
                (player_pos, boxes), move = parents[(player_pos, boxes)]
                path.append(move)
            return path[::-1], len(visited)

        for direction in ['U', 'D', 'L', 'R']:
            new_game = GameWrapper(game_map)
            new_game.game.set_player_position(player_pos)
            new_game.game.set_box_positions(list(boxes))

            if new_game.game.apply_move(direction):
                new_player_pos = new_game.game.get_player_position()
                new_boxes = tuple(new_game.game.get_box_locations())
                new_state = (new_player_pos, new_boxes)

                g = g_scores[(player_pos, boxes)] + 1  # Cost to reach new state
                h = weight * game.heuristic(new_boxes)  # Heuristic 
                f = g + h

                if new_state not in g_scores or g < g_scores[new_state]:
                    g_scores[new_state] = g
                    counter += 1
                    heapq.heappush(priority_queue, (f, counter, new_state))
                    parents[new_state] = ((player_pos, boxes), direction)

    return None, len(visited)  # No solution found


def solver_weighted_astar(game_map, weight=2):
    return solver_astar(game_map, weight)


## Solve

In [23]:
SOLVERS = {
    "BFS": solver_bfs,
    "DFS": solver_dfs,
    "IDS": solver_ids,
    "A*": solver_astar,
    "Weighted A*": solver_weighted_astar
}


In [24]:
def solve(map, method):  
    
    if not is_valid_input(map, map_file_indices, method, SOLVERS):
        return
    
    file_name = "map" + str(map) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    
    start_time = time.time()
    moves, numof_visited_states = SOLVERS[method](game_map)
    end_time = time.time()
    print(f"{method} took {round(end_time - start_time, 2)} seconds on map {map} and visited {numof_visited_states} states.")
    
    if moves is None:
        print("No Solution Found!")
    else:
        print(f"{len(moves)} moves were used: {moves}")
            

### Answer of the Question 9:

### BFS Tests:

In [28]:
solve(6, "BFS") # Solve map 1 using BFS

BFS took 0.67 seconds on map 6 and visited 28413 states.
38 moves were used: ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'D', 'L', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'L', 'U', 'R', 'D', 'R', 'R', 'R', 'R', 'R', 'R']


In [ ]:
solve(7,"BFS")

Above testcase took more than a minute.

In [31]:
solve(8,"BFS")

BFS took 0.03 seconds on map 8 and visited 4273 states.
14 moves were used: ['U', 'U', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'R', 'D', 'R']


In [ ]:
solve(9,"BFS")

Above testcase took more than 1 minute.

In [33]:
solve(10,"BFS")

BFS took 3.45 seconds on map 10 and visited 218537 states.
46 moves were used: ['R', 'R', 'R', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'L', 'L', 'L', 'U', 'L', 'D', 'R', 'U', 'U', 'U', 'L', 'D', 'R', 'D', 'L', 'D', 'R', 'R', 'D', 'R', 'U', 'L', 'U', 'R', 'U', 'R', 'D', 'D', 'R', 'D', 'L', 'L', 'L', 'L']


### DFS Tests:

In [34]:
solve(6,"DFS")

DFS took 0.05 seconds on map 6 and visited 5291 states.
251 moves were used: ['R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R

In [ ]:
solve(7,"DFS")

Above testcase took more than one minute.

In [36]:
solve(8,"DFS")

DFS took 0.33 seconds on map 8 and visited 28045 states.
1224 moves were used: ['R', 'R', 'R', 'R', 'D', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'D', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'U', 'R', 'R', 'R', 'R', 'D', 'L', 'D', 'R', 'D', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'D', 'R', 'D', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 

In [ ]:
solve(9,"DFS")

Above testcase took more than a minute.

In [38]:
solve(10,"DFS")

DFS took 31.35 seconds on map 10 and visited 414897 states.
12702 moves were used: ['R', 'R', 'R', 'R', 'R', 'D', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'U', 'L', 'L', 'L', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'L', 'L', 'L', 'L', 'D', 'R', 'R', 'R', 'R', 'R', 'U', 'U', 'L', '

### IDS Tests:

In [39]:
solve(6,"IDS")

IDS took 3.93 seconds on map 6 and visited 5430 states.
85 moves were used: ['U', 'U', 'U', 'U', 'U', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'R', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'R', 'U', 'R', 'U', 'R', 'U', 'U', 'U', 'R', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'L']


In [40]:
solve(7,"IDS")

IDS took 48.74 seconds on map 7 and visited 242326 states.
44 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'L', 'U', 'R', 'D', 'R', 'U', 'L', 'D', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R', 'R', 'U', 'R', 'R', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L']


In [41]:
solve(8,"IDS")

IDS took 0.29 seconds on map 8 and visited 179 states.
22 moves were used: ['U', 'U', 'U', 'R', 'D', 'D', 'U', 'U', 'L', 'D', 'D', 'D', 'R', 'R', 'D', 'R', 'U', 'U', 'R', 'D', 'D', 'R']


In [ ]:
solve(9,"IDS")

Above testcase took more than a minute.

In [ ]:
solve(10,"IDS")

Above testcase took more than a minute.

### A* Tests:

In [44]:
solve(6,"A*")

A* took 2.74 seconds on map 6 and visited 7036 states.
34 moves were used: ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'R']


In [45]:
solve(7,"A*")

A* took 13.24 seconds on map 7 and visited 58881 states.
34 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R']


In [46]:
solve(8,"A*")

A* took 0.44 seconds on map 8 and visited 952 states.
19 moves were used: ['U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'D', 'D', 'R']


In [ ]:
solve(9,"A*")

above testcase took more than a minute.

In [48]:
solve(10,"A*")

A* took 2.78 seconds on map 10 and visited 7077 states.
29 moves were used: ['U', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'U', 'U', 'L', 'L', 'L', 'D', 'R', 'U', 'U', 'L', 'U', 'L', 'D']


### Weighted A*:

In [49]:
solve(6,"Weighted A*")

Weighted A* took 2.13 seconds on map 6 and visited 3280 states.
34 moves were used: ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'R', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'D', 'L', 'R', 'R', 'R', 'R', 'R', 'R', 'R']


In [50]:
solve(7,"Weighted A*")

Weighted A* took 5.3 seconds on map 7 and visited 9495 states.
34 moves were used: ['R', 'U', 'R', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'R', 'U', 'U', 'U', 'U', 'L', 'L', 'L', 'R', 'D', 'R', 'D', 'R', 'D', 'D', 'L', 'L', 'D', 'L', 'L', 'U', 'U', 'D', 'R']


In [51]:
solve(8,"Weighted A*")

Weighted A* took 0.06 seconds on map 8 and visited 26 states.
19 moves were used: ['U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'R', 'D', 'D', 'R']


In [ ]:
solve(9,"Weighted A*")

Above testcase took more than a minute.

In [53]:
solve(10,"Weighted A*")

Weighted A* took 0.8 seconds on map 10 and visited 2537 states.
29 moves were used: ['U', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'D', 'L', 'L', 'L', 'U', 'R', 'R', 'R', 'U', 'U', 'L', 'L', 'L', 'D', 'R', 'U', 'U', 'L', 'U', 'L', 'D']


In [27]:
def solve_all():
    for map in range(min(map_file_indices), max(map_file_indices) + 1):
        for method in SOLVERS.keys():
            solve(map, method)
            

In [32]:
solve_all() # Solve all maps using all methods

BFS took 0.0 seconds on map 1 and visited 5 states.
No Solution Found!
DFS took 0.0 seconds on map 1 and visited 5 states.
No Solution Found!
IDS took 0.0 seconds on map 1 and visited 10 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
A* took 0.02 seconds on map 1 and visited 44 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
Weighted A* took 0.01 seconds on map 1 and visited 41 states.
7 moves were used: ['U', 'D', 'D', 'U', 'L', 'R', 'R']
BFS took 0.0 seconds on map 2 and visited 5 states.
No Solution Found!
DFS took 0.0 seconds on map 2 and visited 5 states.
No Solution Found!
IDS took 0.0 seconds on map 2 and visited 7 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
A* took 0.0 seconds on map 2 and visited 21 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
Weighted A* took 0.0 seconds on map 2 and visited 16 states.
6 moves were used: ['L', 'U', 'D', 'D', 'U', 'L']
BFS took 0.0 seconds on map 3 and visited 37 states.
No Solution Fou

KeyboardInterrupt: 

#Questions:


3. Q: Explain how you would model the problem, including the definition of actions, states, goal states, etc.

A: State: A tuple (player_pos, boxes_pos), where:
    player_pos: The player's current position (row, col).
    boxes_pos: A tuple of positions for all boxes ((row1, col1), (row2, col2), ...).
    Actions: The player can move in four directions (U, D, L, R).
    Goal State: A state where all boxes are on their corresponding goals.
    Initial State: The starting positions of the player and boxes.
    Transitions: For each action, generate a new state by updating the player's position and, if applicable, the box's position.

4. Q: How can this problem be improved, and how can the search space be optimized? Should we expand all possible states in the next step of the search algorithms? What are the implications of doing so? What measures can be taken to improve this problem?

A: Improvements:
    Use heuristics to guide the search (e.g., Manhattan distance for A*).
    Prune invalid or redundant states (e.g., states where boxes are stuck against walls).
    Limit the depth of exploration (e.g., in IDS).
    Expanding All States:
    Expanding all possible states at each step can lead to exponential growth in the search space, making the algorithm slow and memory-intensive.
    This is especially problematic for large maps or complex puzzles.
    Optimization Measures:
    Use admissible heuristics to prioritize promising states.
    Implement state caching to avoid revisiting the same state.
    Use iterative deepening to balance memory usage and completeness.